# SynTool tutorial: retrosynthesis planning

<font size="4">This tutorial explains how to perform retrosynthesis planning in SynTool using various planning algorithm settings</font>

## 1. Download input data

The input data will be downloaded from the Google disk to the current location (./). The planning data directory includes.

 - `uspto_reaction_rules.pickle` - the reaction rules extracted from the USPTO dataset.
 - `filtering_policy_network.ckpt` - the trained filtering policy network.
 - `ranking_policy_network.ckpt` - the trained ranking policy network.
 - `value_network.ckpt` - the trained value network network.
 - `building_blocks.smi` -  a set of building block molecules used in value neural network tuning (not considered in this tutorial) and in retrosynthesis planning.

In [1]:
import os
import shutil
import gdown

In [3]:
remote_id = "1HFL8yT5i2wE82lNqB88wZ5OM2D9AsvXH"
data_archive = os.path.join("syntool_planning_data.zip")
gdown.download(output=data_archive, id=remote_id, quiet=False)

shutil.unpack_archive(data_archive)
os.remove(data_archive)
if os.path.exists('__MACOSX'):
    shutil.rmtree('__MACOSX')

## 2. SynTool planning algorithms

Retrosynthesis planning in SynTool is based on the Monte-Carlo Tree Search (MCTS) algorithm, which has several configurations depending on the type of expansion/evaluation function and search strategy. In this section, we compare some planning configurations in SynTool. 

- For planning performance comparison, we take a set of 100 target molecules with a synthetic accessibility score (calculated with RDKit) between 2.5 and 3.5 (medium complexity).
 
- For all planning configurations we fix (except section 3.4) the maximum amount of search time, the maximum number of iterations, and the maximum depth of the search tree, as well as a set of reaction rules and building blocks.

In [5]:
import pandas as pd

from SynTool.utils.config import TreeConfig
from SynTool.utils.config import PolicyNetworkConfig
from SynTool.mcts.search import tree_search

In [6]:
targets_path = 'targets_with_sascore_2.5_3.5.smi'

In [7]:
search_config = {"max_time":120,
                 "max_iterations":100,
                 "max_depth":9, 
                 "silent":True}

reaction_rules_path = 'syntool_planning_data/uspto_reaction_rules.pickle'
building_blocks_path = 'syntool_planning_data/building_blocks.smi'

In [8]:
def extract_stats(res_df, method_name=None):
    
    df = pd.DataFrame([{'METHOD':method_name,
                        'SOLVED_TARGETS':(res_df["num_routes"] > 0).sum(),
                        'AVERAGE_NUMBER_OF_NODES': int(res_df["num_nodes"].mean()),
                        'AVERAGE_NUMBER_OF_ROUTES': int(res_df["num_routes"].mean()),
                        'AVERAGE_NUMBER_OF_ITERATIONS': int(res_df["num_iter"].mean()),
                        'AVERAGE_TIME': round(res_df["search_time"].mean(), 1)}])
    return df

### 2.1. Ranking vs Filtering policy network

The tree nodes in MCTS are expanded by an expansion function approximated by a policy graph neu-ral network.
The policy network is composed of two parts: molecular representation and reaction rule prediction parts.
In the representation part, the molecular graph is converted to a single vector by graph convolutional layers.
The training set structure and the prediction part architecture depend on the type of policy network,
particularly the ranking or filtering policy network.

**Ranking policy network**. The training dataset for ranking policy network consists of pairs of reactions and
corresponding reaction rules extracted from it. The products of the reaction are transformed to the CGR encoded
as a molecular graph with the one-hot encoded label vector where the positive label corresponds to the reaction rule.
The prediction part is terminated with the softmax function generating the “probability of successful application” of
each reaction rule to a given input molecular graph, which can be used for the reaction rules “ranking”.

**Filtering policy network**. The training dataset for the filtering policy is formed by the application of all
reaction rules to the training molecules. The labels vector is filled with positive labels in positions corresponding
to the successfully applied reaction rules. The prediction part of the filtering policy is formed from two linear layers
with a sigmoid function that assigns the probabilities for the “regular”, as well as “priority” reac-tion rules
(cyclization reaction rules). These two vectors are then combined with a coefficient α ranging from 0 to 1.
This approach ensures that the priority reaction rules receive the highest score, followed by other regular reaction rules.
The filtering policy network requires much more computational resources for the generating of the training dataset than
the ranking policy but can be used with any set of reaction rules because the original reaction dataset is not needed.
This allows for the portability of reaction rules extracted with another software from any source of reaction data.

#### Search with ranking policy newtwork

In [9]:
search_config["evaluation_type"] = "rollout"
search_config["search_strategy"] = "expansion_first"
policy_config = PolicyNetworkConfig(weights_path='syntool_planning_data/ranking_policy_network.ckpt')

tree_search(targets_path=targets_path,
            search_config=search_config,
            policy_config=policy_config,
            reaction_rules_path=reaction_rules_path,
            building_blocks_path=building_blocks_path,
            value_network_path=None,
            results_root="search_results_ranking")

Number of target molecules processed: 100 [51:16]

Number of solved target molecules: 32


#### Search with filtering policy newtwork

In [11]:
# search_config["evaluation_type"] = "rollout"
# search_config["search_strategy"] = "expansion_first"
# policy_config = PolicyNetworkConfig(weights_path='syntool_planning_data/filtering_policy_network.ckpt')

# tree_search(targets_path=targets_path,
#             search_config=search_config,
#             policy_config=policy_config,
#             reaction_rules_path=reaction_rules_path,
#             building_blocks_path=building_blocks_path,
#             value_network_path=None,
#             results_root="search_results_filtering")

#### Exapansion results comparison

In [12]:
res_ranking = pd.read_csv('search_results_ranking/tree_search_stats.csv')

pd.concat([extract_stats(res_ranking, 'Ranking')])

,METHOD,SOLVED_TARGETS,AVERAGE_NUMBER_OF_NODES,AVERAGE_NUMBER_OF_ROUTES,AVERAGE_NUMBER_OF_ITERATIONS,AVERAGE_TIME
0,Ranking,32,983,15,99,29.6


### 2.2. Rollout vs Value Network evaluation

**Node evaluation**. During the evaluation step, the value function (or evaluation function) is used to estimate the
retrosynthetic feasibility of newly created nodes. In SynTool, there are three types of evaluation functions implemented:

 - `random function` - assigns a random value between 0 and 1 to the new node. Mostly used as a baseline.

 - `rollout function`  - default evaluation type in MCTS. In the current implementation it does a series of node expansions until it reaches some stope criterion (maximum simulation depth, discovered retrosynthetic route, etc.). Based on the simulation results it assigns the value between (-1 and 1) to the new node.

 - `value network` - instantly predicts the value between 0 and 1. The value neural network is trained on the data from planning simulations (performed with the previous version of the value network) including examples with precursors leading to the solutions and those which are part of the unsuccessful routes.

**Value network tuning**. The training set for the value neural network is generated from the simulations of planning sessions.
In the first iteration, the value network is initialized with random weights and is used for the initial retrosynthesis
planning session for N target molecules. Then, precursors that were part of a successful retrosynthesis path leading
to building block molecules are labeled with a positive label, and precursors that did not lead to building blocks are
labeled with a negative label. This generated training data is used to train the value network to better recognize precursors
leading to possible successful retrosynthetic paths. The trained value network is used in the next iteration of the simulated
planning session alternat-ed by the retraining of the value network until it reaches the acceptable accuracy of predictions.

In [13]:
search_config["search_strategy"] = "expansion_first"
policy_config = PolicyNetworkConfig(weights_path='syntool_planning_data/ranking_policy_network.ckpt')

#### Search with random evaluation

In [14]:
search_config["evaluation_type"] = "random"

tree_search(targets_path=targets_path,
            search_config=search_config,
            policy_config=policy_config,
            reaction_rules_path=reaction_rules_path,
            building_blocks_path=building_blocks_path,
            value_network_path=None,
            results_root="search_results_random")

Number of target molecules processed: 100 [28:04]

Number of solved target molecules: 32


#### Search with rollout evaluation

In [15]:
search_config["evaluation_type"] = "rollout"

tree_search(targets_path=targets_path,
            search_config=search_config,
            policy_config=policy_config,
            reaction_rules_path=reaction_rules_path,
            building_blocks_path=building_blocks_path,
            value_network_path=None,
            results_root="search_results_rollout")

Number of target molecules processed: 100 [50:05]

Number of solved target molecules: 32


#### Search with value network evaluation

In [16]:
search_config["evaluation_type"] = "gcn"
value_network_path = 'syntool_planning_data/value_network.ckpt'

tree_search(targets_path=targets_path,
            search_config=search_config,
            policy_config=policy_config,
            reaction_rules_path=reaction_rules_path,
            building_blocks_path=building_blocks_path,
            value_network_path=value_network_path,
            results_root="search_results_value")

Number of target molecules processed: 100 [35:41]

Number of solved target molecules: 25


#### Evaluation results comparison

In [17]:
res_random = pd.read_csv('search_results_random/tree_search_stats.csv')
res_rollout = pd.read_csv('search_results_rollout/tree_search_stats.csv')
res_value = pd.read_csv('search_results_value/tree_search_stats.csv')

pd.concat([extract_stats(res_random, 'Random'),
           extract_stats(res_rollout, 'Rollout'),
           extract_stats(res_value, 'Value network')])

,METHOD,SOLVED_TARGETS,AVERAGE_NUMBER_OF_NODES,AVERAGE_NUMBER_OF_ROUTES,AVERAGE_NUMBER_OF_ITERATIONS,AVERAGE_TIME
0,Random,32,975,12,99,16.1
0,Rollout,32,983,15,99,29.0
0,Value network,25,1106,9,99,20.9


### 3.3. Expansion-first vs Evaluation-first search strategy

The retrosynthesis planning in SynTool is executed with the MCTS algorithm. The nodes in the MCTS algorithm are expanded
by the expansion function predicting reaction rules applicable to the current precursor and evaluated by
the evaluation function navigating the tree exploration in the promising directions. The tree search is limited
by tree parameters: number of iterations, time of the search, and size of the tree (total number of nodes).
Retrosynthesis planning in SynTool can be performed using two search strategies:
the evaluation-first and the expansion-first strategy.

**Expansion-first strategy.** In the expansion-first strategy, each newly created node is assigned a predefined constant value.
This approach is characterized by a more stochastic selection of nodes for expansion but allows for a reduction in the
computational resources.

**Evaluation-first strategy.** In the evaluation-first strategy, each newly created node immediately is evaluated with
the evaluation function, which allows for more exhaustive tree exploration. Although the node evaluation in the
evaluation-first strategy imposes an additional computational overhead, this problem can be overcome by the application
of fast evaluation functions, such as one approximated by a value neural network.

In [18]:
search_config["evaluation_type"] = "gcn"
value_network_path = 'syntool_planning_data/value_network.ckpt'
policy_config = PolicyNetworkConfig(weights_path='syntool_planning_data/ranking_policy_network.ckpt')

#### Search with expansion-first strategy

In [19]:
search_config["search_strategy"] = "expansion_first"

tree_search(targets_path=targets_path,
            search_config=search_config,
            policy_config=policy_config,
            reaction_rules_path=reaction_rules_path,
            building_blocks_path=building_blocks_path,
            value_network_path=value_network_path,
            results_root="search_results_expansion")

Number of target molecules processed: 100 [35:33]

Number of solved target molecules: 25


#### Search with evaluation-first strategy

In [20]:
search_config["search_strategy"] = "evaluation_first"

tree_search(targets_path=targets_path,
            search_config=search_config,
            policy_config=policy_config,
            reaction_rules_path=reaction_rules_path,
            building_blocks_path=building_blocks_path,
            value_network_path=value_network_path,
            results_root="search_results_evaluation")

Number of target molecules processed: 100 [1:04:30]

Number of solved target molecules: 40


#### Search strategy results comparison

In [21]:
res_exp = pd.read_csv('search_results_expansion/tree_search_stats.csv')
res_eva = pd.read_csv('search_results_evaluation/tree_search_stats.csv')

pd.concat([extract_stats(res_exp, 'Expansion-first'),
           extract_stats(res_eva, 'Evaluation-first')])

,METHOD,SOLVED_TARGETS,AVERAGE_NUMBER_OF_NODES,AVERAGE_NUMBER_OF_ROUTES,AVERAGE_NUMBER_OF_ITERATIONS,AVERAGE_TIME
0,Expansion-first,25,1106,9,99,20.8
0,Evaluation-first,40,1192,26,98,37.4
